Data Preparation  
Split the data into training and testing sets using an 80-20 ratio.  
Scale the numerical features to ensure the algorithms perform effectively.  

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

soil_data = pd.read_csv('clean_data_soil_crops.csv')


In [3]:
#el objetivo es lo que se va a clasificar gracias a las características
X = soil_data[['pH', 'Moisture', 'Nitrogen', 'SoilType']]  # Características
y = soil_data['Crop']  # Objetivo 

In [4]:
#convertir variables categóricas en variables binarias (hot encoding)
#Los modelos de machine learning generalmente no pueden trabajar directamente con variables categóricas de tipo texto
X = pd.get_dummies(X, columns=['SoilType'], drop_first=True)


In [5]:
# Dividir los datos en conjuntos de entrenamiento y prueba (80-20)
#el random state es una semilla de numeros aleatorios
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=9)


In [7]:
#escalar las caracteristicas
#estandariza las características numéricas eliminando la media y dividiendo entre la desviación estándar.
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train[['pH', 'Moisture', 'Nitrogen']])
X_test_scaled = scaler.transform(X_test[['pH', 'Moisture', 'Nitrogen']])

In [8]:
# Reemplazar las columnas escaladas en los conjuntos de entrenamiento y prueba
X_train[['pH', 'Moisture', 'Nitrogen']] = X_train_scaled
X_test[['pH', 'Moisture', 'Nitrogen']] = X_test_scaled

## Implementcaión del modelo

se escogió el algoritmo de bosques aleatorios


In [30]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
#parametros
modelo = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=9)

#entrena el modelo (model) usando los datos de entrenamiento (X_train_scaled y y_train)
modelo.fit(X_train_scaled, y_train)

accuracy = modelo.score(X_test_scaled, y_test)
accuracy


0.9743589743589743

In [31]:
# Predicciones y métricas
y_pred = modelo.predict(X_test_scaled)
print("Matriz de confusión:")
print(confusion_matrix(y_test, y_pred))
print("Reporte de clasificación:")
print(classification_report(y_test, y_pred, zero_division=1))


Matriz de confusión:
[[38  0]
 [ 1  0]]
Reporte de clasificación:
              precision    recall  f1-score   support

       Maize       0.97      1.00      0.99        38
        Rice       1.00      0.00      0.00         1

    accuracy                           0.97        39
   macro avg       0.99      0.50      0.49        39
weighted avg       0.98      0.97      0.96        39



In [32]:
nuevos_datos = [[4.5, 88, 8]]  # PH, Moisture, Nitrogen
nuevos_datos_scaled = scaler.transform(nuevos_datos)
prediccion = modelo.predict(nuevos_datos_scaled)
print("El modelo predice que el cultivo recomendado es:", prediccion[0])

El modelo predice que el cultivo recomendado es: Maize


c:\Users\Alejando\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


el modelo no parece reaccionar bien con el algoritmo de entrenamiento, usaremos KNN

In [66]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=6)#numero de vecinos 

# Entrenar el modelo con el conjunto de entrenamiento
knn.fit(X_train_scaled, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = knn.predict(X_test_scaled)

accuracy = accuracy_score(y_test, y_pred)
print(f"Exactitud del modelo KNN: {accuracy * 100:.4f}%")


Exactitud del modelo KNN: 97.4359%


In [67]:
conf_matrix = confusion_matrix(y_test, y_pred)
print("Matriz de confusión:")
print(conf_matrix)

Matriz de confusión:
[[38  0]
 [ 1  0]]


In [68]:
from sklearn.model_selection import GridSearchCV

# Definir los parámetros a probar
param_grid = {'n_neighbors': range(1, 21)}  # Probar valores entre 1 y 20

# Realizar la búsqueda en grid
grid_search = GridSearchCV(KNeighborsClassifier(), param_grid, cv=5)  # 5-fold cross-validation
grid_search.fit(X_train_scaled, y_train)

# Mostrar el mejor valor de n_neighbors
print(f"Mejor valor de n_neighbors: {grid_search.best_params_['n_neighbors']}")



Mejor valor de n_neighbors: 4


In [72]:
# Entrenar el modelo con todos los datos (X_scaled y y)
knn_final = KNeighborsClassifier(n_neighbors=grid_search.best_params_['n_neighbors'])  # Mejor valor de n_neighbors
knn_final.fit(X_train_scaled, y_train)

# Hacer predicciones en nuevos datos (por ejemplo, una nueva muestra)
new_data = [[6.5, 80, 2.8]]  # Ejemplo de nuevos datos  PH, Moisture, Nitrogen
new_data_scaled = scaler.transform(new_data)  # Asegúrate de escalar los nuevos datos también
prediction = knn_final.predict(new_data_scaled)

print(f"Predicción para los nuevos datos: {prediction[0]}")

Predicción para los nuevos datos: Maize


c:\Users\Alejando\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


probaremos mejor con svc

In [90]:
from sklearn.svm import SVC

# Train SVM model
svm = SVC(kernel='linear')
svm.fit(X_train_scaled, y_train)

new_data = [[4.5, 85, 8.5]]  # Ejemplo de nuevos datos  PH, Moisture, Nitrogen
new_data_scaled = scaler.transform(new_data)  # Asegúrate de escalar los nuevos datos también

# Predict on test set
y_pred_svm = svm.predict(X_test_scaled)

print(f"Predicción para los nuevos datos: {prediction[0]}")




Predicción para los nuevos datos: Maize


c:\Users\Alejando\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


nuestro modelo de datos esta muy influenciado por el amplio rango del maiz en el dataset, afectando nuestro modelo ya que predice en su mayoria maiz 